## Imports

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 

In [2]:
chart = pd.read_csv('../regional-global-daily-latest.csv', header = 1)
chart.head()

,Position,Track Name,Artist,Streams,URL
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...


In [3]:
song_uris = []
for url in chart['URL']:
     song_uris.append(url[31:])
chart['song_uri'] = song_uris

In [4]:
chart.head()

,Position,Track Name,Artist,Streams,URL,song_uri
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...,0bYg9bo50gSsH3LtXe2SQn
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2FRnf9qhLbvw8fu4IBXx78
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...,0lizgQ7Qw35od7CYaoMBZb
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...,0lLdorYw7lVrJydTINhWdI


## Credentials 

In [5]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group if you're signing in for the first time
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    track_audio_features = []
    artist_uri = []
    artist_name = []
    loops = 0
    while 20* (loops+1) <= len(song_uris):
        track_audio_features.extend(sp.audio_features(song_uris[20*loops :20*(loops+1)]))
        temp= [i for i in sp.tracks(song_uris[20*loops :20*(loops + 1)])['tracks'] ]
        for index in range(len(temp)):
            artist_uri.append([artist['id'] for artist in temp[index]['artists']])
            artist_name.append([artist['name'] for artist in temp[index]['artists']])
        loops += 1
    
    audio_features = pd.DataFrame(track_audio_features)
    chart['artist_uri'] = artist_uri
    df = chart.merge(audio_features, left_on = 'song_uri', right_on= 'id')
    
else:
    print("Can't get token for", username)

In [7]:
df['collaborators'] = df['artist_uri'].apply(lambda x : len(x))

In [8]:
df[df['collaborators']>1]

,Position,Track Name,Artist,Streams,URL,song_uri,artist_uri,danceability,energy,key,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,collaborators
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49,"[4q3ewBCX7sLwd24euuV69X, 0EFisYRi20PTADoJrifHrz]",0.731,0.573,4,...,0.145,109.928,audio_features,4MzXwWMhyBbmu6hOcLVD49,spotify:track:4MzXwWMhyBbmu6hOcLVD49,https://api.spotify.com/v1/tracks/4MzXwWMhyBbm...,https://api.spotify.com/v1/audio-analysis/4MzX...,205090,4,2
8,9,Mood (feat. iann dior),24kGoldn,3518616,https://open.spotify.com/track/3tjFYV6RSFtuktY...,3tjFYV6RSFtuktYl3ZtYcq,"[6fWVd57NKTalqvmjRd2t8Z, 6ASri4ePR7RlsvIQgWPJpS]",0.700,0.722,7,...,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4,2
12,13,LA NOCHE DE ANOCHE,Bad Bunny,3189182,https://open.spotify.com/track/2XIc1pqjXV3Cr2B...,2XIc1pqjXV3Cr2BQUGNBck,"[4q3ewBCX7sLwd24euuV69X, 7ltDVBr6mKbRvohxheJ9h1]",0.856,0.618,7,...,0.391,81.993,audio_features,2XIc1pqjXV3Cr2BQUGNBck,spotify:track:2XIc1pqjXV3Cr2BQUGNBck,https://api.spotify.com/v1/tracks/2XIc1pqjXV3C...,https://api.spotify.com/v1/audio-analysis/2XIc...,203201,4,2
15,16,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,2865446,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,7hxHWCCAIIxFLCzvDgnQHX,"[6MPCFvOQv5cIGfw3jODMF0, 2hlmm7s2ICUX0LVIhVFlZ...",0.800,0.658,1,...,0.462,140.042,audio_features,7hxHWCCAIIxFLCzvDgnQHX,spotify:track:7hxHWCCAIIxFLCzvDgnQHX,https://api.spotify.com/v1/tracks/7hxHWCCAIIxF...,https://api.spotify.com/v1/audio-analysis/7hxH...,195429,4,4
17,18,Monster (Shawn Mendes & Justin Bieber),Shawn Mendes,2815999,https://open.spotify.com/track/1HbA4N1MiOsPthA...,1HbA4N1MiOsPthALesGFR1,"[7n2wHs1TKAczGzO7Dd2rGr, 1uNFoZAHBGtllmzznpCI3s]",0.640,0.392,2,...,0.502,145.838,audio_features,1HbA4N1MiOsPthALesGFR1,spotify:track:1HbA4N1MiOsPthALesGFR1,https://api.spotify.com/v1/tracks/1HbA4N1MiOsP...,https://api.spotify.com/v1/audio-analysis/1HbA...,178994,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,183,Go Crazy,Chris Brown,806597,https://open.spotify.com/track/1IIKrJVP1C9N7iP...,1IIKrJVP1C9N7iPtG6eOsK,"[7bXgB6jMjp9ATFy66eO08Z, 50co4Is1HCEo8bhOyUWKpn]",0.755,0.578,0,...,0.581,94.148,audio_features,1IIKrJVP1C9N7iPtG6eOsK,spotify:track:1IIKrJVP1C9N7iPtG6eOsK,https://api.spotify.com/v1/tracks/1IIKrJVP1C9N...,https://api.spotify.com/v1/audio-analysis/1IIK...,176960,4,2
188,189,Hello (feat. A Boogie Wit da Hoodie),Pop Smoke,791087,https://open.spotify.com/track/2r6OAV3WsYtXuXj...,2r6OAV3WsYtXuXjvJ1lIDi,"[0eDvMgVFoNV3TpwtrVCoTj, 31W5EY0aAly4Qieq6OFu6I]",0.905,0.647,10,...,0.367,130.970,audio_features,2r6OAV3WsYtXuXjvJ1lIDi,spotify:track:2r6OAV3WsYtXuXjvJ1lIDi,https://api.spotify.com/v1/tracks/2r6OAV3WsYtX...,https://api.spotify.com/v1/audio-analysis/2r6O...,190534,4,2
189,190,Carol of the Bells,Mykola Dmytrovych Leontovych,786594,https://open.spotify.com/track/4tHqQMWSqmL6YjX...,4tHqQMWSqmL6YjXwsqthDI,"[6lBN3Tzt61u8i9AJoAhKzF, 3dRfiJ2650SZu6GbydcHNb]",0.418,0.106,8,...,0.800,46.718,audio_features,4tHqQMWSqmL6YjXwsqthDI,spotify:track:4tHqQMWSqmL6YjXwsqthDI,https://api.spotify.com/v1/tracks/4tHqQMWSqmL6...,https://api.spotify.com/v1/audio-analysis/4tHq...,85267,4,2
191,192,Santa Baby (with Henri René & His Orchestra),Eartha Kitt,783910,https://open.spotify.com/track/1foCxQtxBweJtZm...,1foCxQtxBweJtZmdxhEHVO,"[1AwO9pWEBSBoWdEZu28XDC, 52G7KZspBD4cCmuRDYRAtx]",0.700,0.139,8,...,0.490,84.485,audio_features,1foCxQtxBweJtZmdxhEHVO,spotify:track:1foCxQtxBweJtZmdxhEHVO,https://api.spotify.com/v1/tracks/1foCxQtxBweJ...,https://api.spotify.com/v1/audio-analysis/1foC...,206067,4,2


In [ ]:
df.drop(['URL', 'song_uri', 'type', 'uri', 'track_href'])

In [ ]:
df.columns